In [589]:
import pandas as pd;
import numpy as np;
from glob import glob;
import time;
import datetime;
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import pickle
from  sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


In [590]:
print(datetime.datetime.now().time())
for fil in glob('part-00001*.csv'):    
    perFileDf = pd.DataFrame()
    start_time = time.time()
    print ("Started processing the file : "+ fil)
    for df in pd.read_csv(fil,iterator=True,chunksize=10000000):                
                
        #df["speedFactor"] = (np.sqrt((df['acceleration_x'])*(df['acceleration_x']) + (df['acceleration_y'])*(df['acceleration_y']) + (df['acceleration_z'])*(df['acceleration_z'])))/(10 + df["Speed"])        
        
        df["absAccX"] = np.abs(df['acceleration_x'])
        df["absAccY"] = np.abs(df['acceleration_y']) - 9.8
        df["absAccZ"] = np.abs(df['acceleration_z'])

        df["absGyroX"] = np.abs(df['gyro_x'])
        df["absGyroY"] = np.abs(df['gyro_y'])
        df["absGyroZ"] = np.abs(df['gyro_z'])
        
        df["speedFactor"] = (np.sqrt((df['absAccX'])*(df['absAccX']) + (df['absAccY'])*(df['absAccY']) + (df['absAccZ'])*(df['absAccZ'])))/(1 + np.abs(df["Speed"]))        
                                
            
        #print (df["speedFactor"].describe(include='all'))
        #print ("==========")
        newDf=df.groupby('bookingID').agg({'absAccX':'mean', 'absAccY':'mean','absAccZ':'mean','absGyroX':'mean', 'absGyroY':'mean','absGyroZ':'mean','Speed':'mean','speedFactor':'mean'})
        
        #newDf=df.groupby('bookingID').agg({'speedFactor':'mean'})
        newDf = newDf.reset_index()
        #print (newDf.head())
        #print ("***************")
        perFileDf =  perFileDf.append(newDf)       
    
    print ("Done with file : "+ fil)   
    elapsed_time = time.time() - start_time
    print ("Time to process file : " + str(fil)  + " is " + str(elapsed_time))
    start_time = time.time()
    perFileDf.to_csv("df_"+fil)
    elapsed_time = time.time() - start_time
    print ("Time to write Df file : " + "df_"+str(fil)  + " is " + str(elapsed_time))

print(datetime.datetime.now().time())
print("End of phase-1")

19:16:53.781977
Started processing the file : part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Done with file : part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Time to process file : part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv is 14.78363299369812
Time to write Df file : df_part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv is 1.4833166599273682
19:17:10.055443
End of phase-1


In [592]:
print(datetime.datetime.now().time())
consolidatedDf = pd.DataFrame()
for fil in glob('df_part-00001*.csv'):    
    start_time = time.time()
    print ("Started processing the file : "+ fil)
    for df in pd.read_csv(fil,iterator=True,chunksize=10000000):                
        print ("Current Df shape" + str(df.shape))
        consolidatedDf = consolidatedDf.append(df)               
        print ("Consolidated Df shape" + str(consolidatedDf.shape))
    print ("Done processing the file : "+ fil)    
    elapsed_time = time.time() - start_time
    print ("Time to process file : " + str(fil)  + " is " + str(elapsed_time))    

consolidatedDf.to_csv("consolidatedDf.csv")
grpConsolidatedDf =consolidatedDf.groupby('bookingID').agg({'absAccX':'mean', 'absAccY':'mean','absAccZ':'mean','absGyroX':'mean', 'absGyroY':'mean','absGyroZ':'mean','speedFactor':'mean','Speed':'mean'})
#grpConsolidatedDf =consolidatedDf.groupby('bookingID').agg({'speedFactor':'mean'})
print ("After Consolidated Df shape" + str(grpConsolidatedDf.shape))
grpConsolidatedDf.to_csv("grpConsolidatedDf.csv")
print(datetime.datetime.now().time())

print("End of phase-2")

19:22:28.196726
Started processing the file : df_part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Current Df shape(20000, 10)
Consolidated Df shape(20000, 10)
Done processing the file : df_part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
Time to process file : df_part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv is 0.22729754447937012
After Consolidated Df shape(20000, 8)
19:22:31.053978
End of phase-2


In [593]:
grpDf =  pd.read_csv("grpConsolidatedDf.csv")                

In [594]:
grpDf.head()

,bookingID,absAccX,absAccY,absAccZ,absGyroX,absGyroY,absGyroZ,speedFactor,Speed
0,0,0.834060,-0.220720,1.682674,0.038779,0.055264,0.035292,0.552266,8.249537
1,1,0.714013,-0.183361,2.127801,0.017789,0.053492,0.022294,0.906879,7.624494
2,2,0.649514,0.031824,0.620117,0.016479,0.082380,0.019386,0.581923,2.871415
3,4,0.495212,-0.427121,2.710666,0.034396,0.075628,0.043784,0.666747,6.282727
4,6,0.811684,-0.295071,2.417631,0.035863,0.060289,0.045958,1.259021,5.011717


In [595]:
grpDf.shape

(20000, 9)

In [596]:
labelDf = pd.read_csv('labels.csv')

In [597]:
labelDf.head()

,bookingID,label
0,111669149733,0
1,335007449205,1
2,171798691856,0
3,1520418422900,0
4,798863917116,0


In [598]:
labelDf.shape

(20018, 2)

In [599]:
labelDf.drop_duplicates(subset=['bookingID'],inplace=True,keep='first')

In [600]:
labelDf.shape

(20000, 2)

In [601]:
withLabelDf = pd.merge(labelDf,grpDf,on="bookingID")

In [602]:
withLabelDf.head()

,bookingID,label,absAccX,absAccY,absAccZ,absGyroX,absGyroY,absGyroZ,speedFactor,Speed
0,111669149733,0,0.379943,-0.078871,0.612965,0.015233,0.030512,0.017338,0.271823,5.107879
1,335007449205,1,0.879976,-0.279679,1.278360,0.037018,0.094605,0.017074,0.584506,6.275801
2,171798691856,0,0.445844,-0.159671,0.380780,0.031890,0.036026,0.045659,0.078154,16.239362
3,1520418422900,0,0.665027,0.091064,0.780459,0.020005,0.056748,0.021926,0.205818,14.271977
4,798863917116,0,0.921230,-0.193657,1.346567,0.073428,0.074201,0.017453,0.627011,8.178224


In [603]:
labelDf.to_csv("labels_df.csv")

In [604]:
withLabelDf.to_csv("withLabel_df.csv")

### Preprocssing is done, now lets apply the classification algorithm

In [605]:
allColumns = withLabelDf.columns

In [606]:
excludeLabels = ["bookingID","label"]

In [607]:
featuresCol = [eachCol for eachCol in allColumns if eachCol not in excludeLabels]

In [608]:
featuresCol

['absAccX',
 'absAccY',
 'absAccZ',
 'absGyroX',
 'absGyroY',
 'absGyroZ',
 'speedFactor',
 'Speed']

In [609]:
 X_train, X_test, y_train, y_test = train_test_split(withLabelDf[featuresCol],withLabelDf['label'],test_size=0.3, random_state=8912)

In [610]:
X_train.head()

,absAccX,absAccY,absAccZ,absGyroX,absGyroY,absGyroZ,speedFactor,Speed
13974,0.959978,0.176437,1.302765,0.053060,0.072878,0.041217,0.604481,6.948176
17424,3.515598,-1.164928,2.405909,0.024948,0.064291,0.022531,0.970237,12.391443
12953,1.315104,-0.200493,0.737547,0.035408,0.040204,0.045702,0.079440,22.298432
973,0.433644,-0.482072,2.848140,0.098897,0.049179,0.064120,0.520745,10.043010
9036,0.669623,-0.256444,1.173365,0.073072,0.048750,0.024157,0.281680,7.406659


In [611]:
X_test.head()


,absAccX,absAccY,absAccZ,absGyroX,absGyroY,absGyroZ,speedFactor,Speed
11635,1.112467,-0.754865,2.957147,0.074271,0.159827,0.129607,0.767410,5.394711
19216,0.639995,-0.352192,2.228736,0.043849,0.096299,0.039926,0.753423,7.384937
6751,0.721461,0.351166,1.409357,0.013079,0.018415,0.008741,0.094928,18.995200
4248,0.693866,-0.337003,2.727628,0.058503,0.063129,0.037936,0.567766,11.751127
9921,1.711276,-1.597589,5.015348,0.018375,0.044987,0.025918,1.849403,5.647748


In [612]:

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [613]:
X_test[2]


array([0.07214143, 0.2572379 , 0.03558562, 0.00787021, 0.00096088,
       0.00462747, 0.00356838, 0.65399458])

In [614]:
gnb= GaussianNB()

In [615]:
model = gnb.fit(X_train, y_train)

# Make predictions
preds = gnb.predict(X_test)
print(preds)

# Evaluate accuracy
print(accuracy_score(y_test, preds))

[0 0 0 ... 0 0 0]
0.7415


In [622]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of Decision Tree classifier on test set: 0.64


In [623]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

Accuracy of K-NN classifier on training set: 0.79
Accuracy of K-NN classifier on test set: 0.71


In [624]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))
print('Accuracy of LDA classifier on test set: {:.2f}'
     .format(lda.score(X_test, y_test)))

Accuracy of LDA classifier on training set: 0.76
Accuracy of LDA classifier on test set: 0.76


In [625]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(X_test, y_test)))

Accuracy of GNB classifier on training set: 0.74
Accuracy of GNB classifier on test set: 0.74


In [620]:
from sklearn.svm import SVC
svm = SVC(C=10,gamma=1)
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.2f}'
     .format(svm.score(X_test, y_test)))

Accuracy of SVM classifier on training set: 0.76
Accuracy of SVM classifier on test set: 0.76


In [536]:
nfolds = 10
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds,verbose=10,n_jobs= -1)
grid_search.fit(X_train, y_train)
grid_search.best_params_
print(grid_search.best_params_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   44.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  9.2min
C:\Users\pitva01\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short 

{'C': 10, 'gamma': 1}


In [ ]:

def create_baseline():

	model = Sequential()
	model.add(Dense(10, input_dim=10, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [621]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = gnb.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[4249  231]
 [1320  200]]
              precision    recall  f1-score   support

           0       0.76      0.95      0.85      4480
           1       0.46      0.13      0.21      1520

   micro avg       0.74      0.74      0.74      6000
   macro avg       0.61      0.54      0.53      6000
weighted avg       0.69      0.74      0.68      6000



### Lets save the model to file

In [626]:
filename = 'finalized_model.sav'
pickle.dump(lda, open(filename, 'wb'))
 

#  <font color='red'>To run the model against unseen data, you can start executing cells from below</font>

In [627]:

import pandas as pd;
import numpy as np;
from glob import glob;
import time;
import datetime;
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import pickle
from  sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

###  Put the unseen data (against which my model to be evaluated) in unseenData.csv in the same format as that competition file part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv

In [628]:
    df = pd.read_csv("unseenData.csv")                
    perFileDf = pd.DataFrame()
                
    #df["speedFactor"] = (np.sqrt((df['acceleration_x'])*(df['acceleration_x']) + (df['acceleration_y'])*(df['acceleration_y']) + (df['acceleration_z'])*(df['acceleration_z'])))/(10 + df["Speed"])        
    
        
    df["absAccX"] = np.abs(df['acceleration_x'])
    df["absAccY"] = np.abs(df['acceleration_y'])
    df["absAccZ"] = np.abs(df['acceleration_z'])

    df["absGyroX"] = np.abs(df['gyro_x'])
    df["absGyroY"] = np.abs(df['gyro_y'])
    df["absGyroZ"] = np.abs(df['gyro_z'])
    
    df["speedFactor"] = (np.sqrt((df['absAccX'])*(df['absAccX']) + (df['absAccY'])*(df['absAccY']) + (df['absAccZ'])*(df['absAccZ'])))/(1 + np.abs(df["Speed"]))        
                                
    #newDf=df.groupby('bookingID').agg({'absAccX':'mean', 'absAccY':'mean','absAccZ':'mean','absGyroX':'mean', 'absGyroY':'mean','absGyroZ':'mean','speedFactor':'mean'})
    newDf=df.groupby('bookingID').agg({'absAccX':'mean', 'absAccY':'mean','absAccZ':'mean','absGyroX':'mean', 'absGyroY':'mean','absGyroZ':'mean','Speed':'mean','speedFactor':'mean'})
    newDf = newDf.reset_index()
    perFileDf =  perFileDf.append(newDf)

In [629]:
perFileDf.head()


,bookingID,absAccX,absAccY,absAccZ,absGyroX,absGyroY,absGyroZ,Speed,speedFactor
0,0,0.917439,9.588628,1.618996,0.036449,0.063380,0.039019,8.786820,2.961779
1,1,0.590331,9.462895,2.192611,0.018621,0.040373,0.022342,8.564105,3.013037
2,2,0.587353,9.792962,0.792379,0.031081,0.105638,0.019730,3.417660,4.054655
3,4,0.522485,9.433036,2.594428,0.037383,0.078418,0.045463,6.219018,2.573693
4,6,0.628562,9.521509,2.407400,0.038918,0.052217,0.032318,4.386527,5.326274


In [630]:
allColumns = perFileDf.columns
excludeLabels = ["bookingID","label"]

In [631]:
featuresCol = [eachCol for eachCol in allColumns if eachCol not in excludeLabels]

In [633]:
filename = 'finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(perFileDf[featuresCol])
print("This is the model prediction")
print(result)

This is the model prediction
[1 1 0 ... 1 0 0]
